In [11]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

### A Matrix Based Approach to Computing the Output of a Neural Network

To begin let's describe some notation.
1. $w_{jk}^l$ is the weight from neuron $k$ in the $(l-1)^{th}$ layer to neuron $j$ in the $l^{th}$
2. $b_j^l$ is the bias of neuron $j$ in layer $l$
3. $a_j^l$ is the activation of neuron $j$ in layer $l$

With this notation, we can write the activation
$$
a_j^l = \sigma\big(\sum_kw_{jk}^la_k^{(l-1)} + b_j^l\big)
$$

In matrix form, vectorizing $\sigma(\cdot)$
$$
\mathbf{a}^l = \sigma(\mathbf{z}^l) = \sigma\big(\mathbf{W}^l\mathbf{a}^{l-1} + \mathbf{b}^l\big)
$$

Revisiting our cost function, we write
$$
C = \frac{1}{2n}\sum_x\|\mathbf{y}(x) - \mathbf{a}^L(x)\|^2
$$

First, we need to be able to write our cost function as an average of cost functions for individual data points (which we can do in this case): $C = \frac{1}{n}\sum_xC_x$. Second, we must be able to write the cost function as a function of the network output (which we also can do in this case). So for a given input $x$, our cost function evaluation looks like:
$$
C_x = C = \frac{1}{2}\|\mathbf{y} - \mathbf{a}^L\|^2 = \frac{1}{2}\sum_j(y_j - a_j^L)^2
$$

Below is an example of an operation that we will be making consistent use of, called the *Hadamard* or *Schur* product. This is an elementwise multiplication of two vectors rather than the dot product:

In [9]:
a = array([2, 4])
b = array([5, 3])
a*b

array([10, 12])

### The Equations of Backpropagation

We begin by introducing the concept of an *error* $\delta_j^l$ for the $j^{th}$ neuron in layer $l$.

$$
\delta_j^l = \frac{{\partial C}}{{\partial z_j^l}}
$$



**BP Equation 1**: error at the output layer
$$
\delta_j^L = \frac{\partial C}{\partial a_j^L}\sigma'(z_j^L)
$$

The input to neuron $j$ in layer $L$, $z_j^L$ is calculated while computing hte behavior of the network. This is simply passed through the derivative of the sigmoid function, $\sigma'$, and this multiplies the derivative of the cost function with respect to activation $a_j^L$ of neuron $j$ in layer $L$. With our cost function from above for a single input data point,

$$
\frac{\partial C}{\partial a_j^L} = (a_j - y_j)
$$

In matrix notation we have the following,
$$
\mathbf{\delta}^L = \nabla_a{C}\odot\sigma'(\mathbf{z}^L) = (\mathbf{y}-\mathbf{a}^L)\odot\sigma'(\mathbf{z}^L)
$$

or equivalently, without using the Hadamard product,

$$
\mathbf{\delta}^L = \Sigma'(z^L)\nabla_a{C},
$$

where $\Sigma'(z^L)$ is a square matrix with diagonal entries $\Sigma_{jj}' = \sigma'(z_j^L)$

**BP Equation 2**: error $\delta^l$ in terms of error in the next layer $\delta^{l+1}$
$$
\mathbf{\delta}^l = (\mathbf{W}^{l+1})^T\mathbf{\delta}^{l+1} \odot \sigma'(\mathbf{z}^{l})
$$

The weight matrix $\mathbf{W}$, we will recall, has elements $w_{jk}$ equal to the weight from neuron $k$ onto neuron $j$ in the next layer. Thus, a row in $\mathbf{W}$ relates all neurons in the previous layer to one particular neuron $j$ in the next layer. The transpose therefore relates one neuron in layer $l+1$ to each neuron in the previous layer $l$, meaning in the equation above we relate the error in layer $l+1$ to neurons of layer $l$.

In matrix form we can rewrite BP2 as:
$$
\delta_l = \Sigma'(z^l)(\mathbf{W}^{l+1})^T\delta^{l+1}
$$

Then using BP1 and repeatedly applying BP2 we can work the errors through each layer of the network from last to first.

**BP Equation 3**: the change in cost with respect to change in bias in layer $l$ is equal to the error in layer $l$:

$$
\frac{\partial C}{\partial \mathbf{b}^l} = \mathbf{\delta}^l
$$

**BP Equation 4**: the change in cost with respect to change in weight in layer $l$:

$$
\frac{\partial C}{\partial w_{jk}^l} = a_k^{l-1}\mathbf{\delta}_j^l
$$

One consequence we can read out from this is that neurons with small activations lead to only small changes in $\partial C$, so are said to *learn slowly*. More generally, slow learning occurs when either the input neuron has low activation or the output neuron is nearly saturated.

Ultimately all of these equations are the consequence of the chain rule and expressing the partial derivative of the cost function $C$ with respect to one parameter or variable in terms of the partial derivatives with respect to others. It essentially all boils down to: 

$$
\frac{\partial C}{\partial a^l}\frac{\partial a^l}{\partial z^l}\frac{\partial z^l}{\partial w^l},
$$

or $\frac{\partial z^l}{\partial b^l}$.

**The Backpropagation Algorithm**

The algorithm consists of 5 steps.

1. **Input** x: set the corresponding activation, $\mathbf{a}^1$, for the input layer
2. **Feedforward**: for each layer, $l = 2, 3, ... L$, compute $\mathbf{z}^l = \mathbf{W}^l\mathbf{a}^{l-1} + \mathbf{b}^l$ and $\mathbf{a}^l = \sigma(\mathbf{z}^l)$
3. **Output error** $\delta^L$: Compute the error in the last layer, $\delta^L = \mathbf{\Sigma}'(\mathbf{z}^L)\nabla_aC$
4. **Backpropagate the error**: For each layer $l = L-1, L-2, ... 2$, compute $\delta^l = \mathbf{\Sigma}'(\mathbf{z}^L)(\mathbf{W}^{l+1})^T\delta^{l+1}$
5. **Output**: The gradient of the cost function is given by $\frac{\partial C}{\partial w_{jk}^l} = a_k^{l-1}\delta_j^l$ and $\frac{\partial C}{\partial b_j^l} = \delta_j^l$

Now, to update the weights, we need to combine these steps with stochastic gradient descent, which we can do in the following manner:

1. **Input** training data
2. **Set input activation** $a^{x,1}$ for each training example $x$ in the mini-batch
3. Perform steps 2 through 4 above
4. **Gradient descent** For each layer $l=L,L-2,...2$ update weights and biases according to: $w^l \to w^l - \frac{\eta}{m}\sum_x\delta^{x,l}(a^{x,l-1})^T$ and $b^l \to b^l - \frac{\eta}{m}\sum_x\delta^{x,l}$